# 1: Detecting beads for alignment

0) Create projections of DNA (target) and RNA (moving) FISH images for easier visualization.
1) Resave each `msr` file to a `tif` file, where each channel is an individual file, with the suffix`_ch{number}`
2) Detect beads in RNA images, using RS-FISH settings files as input, outputing `merge.csv` list of spots
3) Detect beads in DNA images, using RS-FISH settings files as input, outputing `merge.csv` list of spots

For more details about required in and output please look at the individual notebooks used in the pipeline.

In [ ]:
import os
import queue
import pandas as pd
from pathlib import Path
from datetime import datetime
import papermill as pm
from multiprocessing import Pool
import concurrent.futures


def run_notebook(parameters,notebook_to_run,parameters_common={}):
    
    # change to directory where the notebook is (resolve relative imports)
    os.chdir(Path(notebook_to_run).absolute().parent)
    
    # run notebook
    for parameters_spec in parameters_list:
        parameters = {**parameters_common, **parameters_spec}

        pm.execute_notebook(
           notebook_to_run,
           '/dev/null',
           parameters=parameters)

# 0) Create projections
**In:** folder with 2-channel `.msr` images

**Out:** folder `projections` with maximum inensity projections `.png` images

In [ ]:
parameters_list = [
    {"in_path": "/home/stumberger/ep2024/RNA_DNA_FISH_spot_detection/example/DNAFISH/"},
    {"in_path": "/home/stumberger/ep2024/RNA_DNA_FISH_spot_detection/example/RNAFISH/"}
]

notebook_to_run = "/home/stumberger/ep2024/subscripts/msr_make_projections_2_color.ipynb"

run_notebook(parameters_list,notebook_to_run)

# 1) Split channels and resave `msr` as `tif`
**In:** folder containing folder `raw` with `.msr` images

**Out:** folder `tif` with `.tif` images split by channel

In [ ]:
parameters_list = [
    {"in_path": "/home/stumberger/ep2024/RNA_DNA_FISH_spot_detection/example/DNAFISH/"},
    {"in_path": "/home/stumberger/ep2024/RNA_DNA_FISH_spot_detection/example/RNAFISH/"}
]

notebook_to_run = "/home/stumberger/ep2024/subscripts/resave_msr_as_tif.ipynb"

run_notebook(parameters_list,notebook_to_run)

# 2) Spot detection RNA beads
**In:** 
- `tif_subfolder` containing single channel `tif` images
- `out_subfolder` containing spot detection settings file `chX.txt` generated via RS-FISH Fiji plugin for each channel specified in `channels`
- `acquisition_info.json` file with the sample metadata which you wish to be added to your spots

**Out:**
- folder containing a `csv` file for each provioded `tif`
- `merge.csv` file combining spots and their metadata from all images in the batch
- vis folder with detection visualization for all `tif`s as `png` (good for checking detection results)

In [ ]:
parameters_list = [
    {"in_path": "/home/stumberger/ep2024/RNA_DNA_FISH_spot_detection/example/RNAFISH/"}
]

parameters_common = {"channels": [1],
                    "out_subfolder": "detections_beads/"}

notebook_to_run = "/home/stumberger/ep2024/subscripts/RS-FISH_spot_detection.ipynb"

run_notebook(parameters_list,notebook_to_run,parameters_common)

# 3) Spot detection DNA beads
See 2)

In [ ]:
parameters_list = [
    {"in_path": "/home/stumberger/ep2024/RNA_DNA_FISH_spot_detection/example/DNAFISH/"}
]

parameters_common = {"channels": [2],
                    "out_subfolder": "detections_beads/"}

notebook_to_run = "/home/stumberger/image-analyis-recipes/spot-detection/RS-FISH_spot_detection.ipynb"

run_notebook(parameters_list,notebook_to_run,parameters_common)